<h1> Email Classifier </h1> 

In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
#the spam and valid emails are stored in 2 folders called "easy_ham" and "spam"
#we want to read all the files in these 2 folders and pu the filenames into a list of spam and not spam
ham_filenames = [filename for filename in sorted(os.listdir(os.path.join(os.getcwd(),"easy_ham"))) if len(filename) > 20]
spam_filenames = [filename for filename in sorted(os.listdir(os.path.join(os.getcwd(),"spam"))) if len(filename) > 20]

In [3]:
len(ham_filenames)

2551

In [4]:
len(spam_filenames)

500

To parse emails we will use the email library of Python:

How the email library works for parsing:

The parser takes a serialized version of the email message(a stream of bytes) and converts it to a tree of EmailMessage objects.  The generator takes an EmailMessage and turns it back into a serialized byte stream.

There are 2 parser interfaces available, the Parser API and FeedParser API. The Parser API is most useful when you have the entire text of the message in memory or if the entire message lives in a file on the file system. FeedParser API is useful when you are reading the message from a stream which might block your waiting(reading from a url itself)

In [5]:
#collecting all the parsed ham messages
ham_path = os.path.join(os.getcwd(),"easy_ham")
spam_path = os.path.join(os.getcwd(),"spam")

In [6]:
import email
from email import policy


In [7]:
ham_messages = list()
for filename in ham_filenames:
    with open(os.path.join(ham_path,filename),'rb') as f:
        ham_messages.append(email.parser.BytesParser(policy = email.policy.default).parse(f))


In [8]:
spam_messages = list()
for filename in spam_filenames:
    with open(os.path.join(spam_path,filename),'rb') as f:
        spam_messages.append(email.parser.BytesParser(policy = email.policy.default).parse(f))

In [9]:
print(ham_messages[0].get_content().strip())

Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compiled on fuchsia.cs.mu.OZ.AU at Sun Mar 17 14:55:56 

Emails can have different parts to it, with images, attachments. And these attachments can have emails in them. 

In [10]:
def get_email_structure(email):
    if isinstance(email,str):
        return email
    #get_payload() returns a list if the email is multipart and .is_multipart() = True
    payload = email.get_payload()
    if isinstance(payload,list):
        result = "multipart({})".format(', '.join([get_email_structure(sub_email) for sub_email in payload]))
        return result
    else:
        return email.get_content_type()

In [11]:
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [12]:
structure_counts_ham = structures_counter(ham_messages)
structure_counts_spam = structures_counter(spam_messages)

In [13]:
structure_counts_ham.most_common()

[('text/plain', 2453),
 ('multipart(text/plain, application/pgp-signature)', 72),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, video/mng)', 1),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(text/plain, application/x-pkcs7-signature)', 1),
 ('multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain, application/x-java-applet)', 1)]

In [14]:
structure_counts_spam.most_common()

[('text/plain', 221),
 ('text/html', 181),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 19),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(text/html, text/plain)', 1),
 ('multipart(multipart(text/html), application/octet-stream, image/jpeg)', 1),
 ('multipart(multipart(text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1)]

In [15]:
type_list = list()
temp = [[type_list.append(i) for i in structures] for structures in [structure_counts_ham, structure_counts_spam]]
ham_type_counts = [structure_counts_ham[i] for i in set(type_list)]
spam_type_counts = [structure_counts_spam[i] for i in set(type_list)]

In [16]:
type_counts_df = pd.DataFrame({'Email Type':list(set(type_list)), 'Ham Count':ham_type_counts, 'Spam Count':spam_type_counts})
type_counts_df

,Email Type,Ham Count,Spam Count
0,"multipart(text/plain, video/mng)",1,0
1,"multipart(text/plain, text/html)",8,45
2,multipart/alternative,0,1
3,"multipart(text/html, application/octet-stream)",0,2
4,"multipart(text/plain, multipart(text/plain))",1,0
5,"multipart(text/html, text/plain)",0,1
6,"multipart(text/plain, application/x-java-applet)",1,0
7,multipart(multipart(text/html)),0,5
8,"multipart(text/plain, text/plain)",4,0
9,"multipart(text/plain, multipart(text/plain, te...",1,0


Most valid(ham) emails are text/plain and contain a PGP(Pretty Good Privacy) signature, while Spam emails have a higher amount of HTML messages. 

In [17]:
#create a list of type of each email 
ham_email_type = [get_email_structure(email) for email in ham_messages]
spam_email_type = [get_email_structure(email) for email in spam_messages]

In [18]:
def FindEmailSender(email):
    try:
        return dict(email.items())['From']
    except:
        return "N/A"

In [19]:
#creating list of senders of each email
ham_email_senders = [FindEmailSender(email) for email in ham_messages]
spam_email_senders = [FindEmailSender(email) for email in spam_messages]

In [20]:
#combining the dataset to create a complete dataset for splitting into train and test sets
import numpy as np
X = np.array(ham_messages + spam_messages, dtype = object)
y = np.array([0] * len(ham_messages) + [1]*len(spam_messages))

In [21]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, test_size=0.2)

In [22]:
#turning html in emails into tags we want
#turn head tags into ''
#turn anchor tags into hyper links
#turn all html tags into ''
import re
from html import unescape
def html_to_text(html):
    text = re.sub(r'<head.*?>.*?</head>','',html,flags = re.I|re.M|re.S)
    text = re.sub(r'<a.*?>.*?</a>',' HYPERLINK ',text,flags = re.I|re.M|re.S)
    text = re.sub(r'<.*?>','',text,flags=re.M|re.I|re.S)
    text = re.sub(r'(\s*\n)+','\n',text,flags = re.M|re.I|re.S)
    return unescape(text)

In [23]:
idx_html = [i for i in range(len(X_train)) if get_email_structure(X_train[i]) == 'text/html']

In [24]:
html_to_text(X_train[idx_html[5]].get_content().strip())

'\n HYPERLINK\nCopyright 2002 - All rights reservedIf you would no longer like us\nto contact you or feel that you havereceived this email in error,\nplease  HYPERLINK .'

In [25]:
#check if any EmailMessage objects have more tha one text/html type occuring
html_check = list()
for j in range(len(X_train)):
    i = 0
    for part in X_train[j].walk():
        if part.get_content_type() == 'text/plain':
            i += 1
        if i == 2:
            html_check.append(j)
    
html_check

[41, 805, 942, 1131, 1219, 2033, 2134, 2266]

In [26]:
def email_to_text(email):
    html = None
    for sub_email in email.walk():
        content_type = sub_email.get_content_type()
        if content_type not in ('text/html','text/plain'):
            continue
        try:
            content = sub_email.get_content()
        except:
            content = str(sub_email.get_payload())
        if content_type == 'text/plain':
            return content
        else:
            html = content
    if html:
        return html_to_text(html)

In [137]:
def email_to_text_multipart(email,f,i):
    html = None
    #the first sub_email that is traversed is the main parent email itself. the following will be the children
    for sub_email in email.walk():
        content_type = sub_email.get_content_type()
        f.write('{},{}\n'.format(i,content_type))
        if content_type not in ('text/html','text/plain'):
            #if the parent mail is multipart, use this route
            content_type = email.get_content_type()
            if re.fullmatch(r'multipart/[a-zA-Z]+',content_type) != None:
                textlist = list()
                for sub_email_1 in email.get_payload():
                    try:
                        if sub_email_1.get_content_type() == 'text/plain':
                            textlist.append(sub_email_1.get_content())
                        elif sub_email_1.get_content_type() == 'text/html':
                            textlist.append(html_to_text(sub_email_1.get_content()))
                        else:
                            continue
                    except:
                        if(isinstance(sub_email_1,str)):
                            textlist.append(sub_email_1)
                        else:
                            continue
                if (len(textlist) > 0):
                    return ' '.join(textlist)
            else:
                continue
        try:
            content = sub_email.get_content()
        except:
            content = str(sub_email.get_payload())
        if content_type == 'text/plain':
            return content
        else:
            f.write('{},{}\n'.format(i,content_type))
            html = content
    if html:
        try:
            return html_to_text(html)
        except:
            f.write('{},{}\n'.format(i,html))
            f.close()
            return TypeError

In [221]:
def email_to_text_multipart_2(email,f,i):
    html = None
    email_type = email.get_content_type()
    #j is used as a counter to count on the walk attribute
    textlist = list()
    j = 0
    for sub_email in email.walk():
        j += 1
        sub_type = sub_email.get_content_type()
        f.write('{},{}\n'.format(i,sub_type))
        if sub_type not in ('text/plain','text/html'):
            #if the parent is not text/plain or text/html we want to continue
            continue
        #this is to get the content if the parent is text/html or text/plain
        elif ((j==1) & (sub_type in ('text/plain','text/html')) & (re.fullmatch(r'multipart/[a-zA-Z]+',email_type) == None)):
            try:
                content = sub_email.get_content()
            except:
                content = str(sub_email.get_payload())
            if (sub_type == 'text/plain'):
                f.write('{},2nd Condition,{},{}\n'.format(i,email_type,sub_type))
                return content
            else:
                html = content
                return html_to_text(html)
        elif ((j!=1) & ((r'multipart/[a-zA-Z]+',email_type) != None)):
            f.write('{},MULTIPART,{}\n'.format(i,sub_type))
            try:
                if sub_type == 'text/plain':
                    f.write('{},wrote to textlist,{},{},j:{}\n'.format(i,sub_type,len(textlist),j))
                    textlist.append(sub_email.get_content())
                elif sub_type == 'text/html':
                    f.write('{},wrote to textlist,{},{},j:{}\n'.format(i,sub_type,len(textlist),j))
                    textlist.append(html_to_text(sub_email.get_content()))
                elif(sub_type not in ('text/plain','text/html')):
                    f.write('Not text/html or text/plain \n')
                    continue
            except:
                if(isinstance(sub_email,str)):
                    f.write('{},wrote to textlist as string,{},{}\n'.format(i,sub_type,len(textlist)))
                    textlist.append(sub_email)
                else:
                    continue
    if (len(textlist) > 0):
        return ' '.join(textlist)








In [224]:
#converts emails in the body to "EMAIL"
import re
def convert_email_tags(email_text):
    return re.sub(r'([a-zA-Z0-9\._-]+@[a-zA-Z0-9\._-]+\.[a-zA-Z0-9\._-]+)',' EMAIL ',email_text,flags = re.I|re.S|re.M)

In [225]:
#remove stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stopwords(email_text):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(email_text.strip())
    return [i for i in words if i not in stop_words]

In [226]:
#stem the words to get the root of words
from nltk.stem import PorterStemmer

def stemmer(email_word_list):
    ps = PorterStemmer()
    return [ps.stem(w) for w in email_word_list]

In [227]:
from sklearn.base import BaseEstimator,TransformerMixin
import urlextract
import re
class EmailToWord(BaseEstimator,TransformerMixin):
    def __init__(self,lower_case = True, remove_email = True, remove_punctuation = True, remove_urls = True, stemming = True, remove_stopwords = True, remove_numbers = True):
         self.lower_case =lower_case
         self.remove_email = remove_email
         self.remove_punctuation = remove_punctuation
         self.remove_urls = remove_urls
         self.stemming = stemming
         self.remove_stopwords = remove_stopwords
         self.remove_numbers = remove_numbers
    def fit(self,X,y = None):
        return self
    def transform(self,X,y = None):
        X_transformed = []
        with open('log.txt','w') as f:
            for i,email in enumerate(X):
                text = email_to_text_multipart_2(email,f,i) or ""
                if self.lower_case:
                    text = text.lower()
                if self.remove_urls:
                    url_extractor = urlextract.URLExtract()
                    urls = url_extractor.find_urls(text)
                    for url in urls:
                        text = text.replace(url,' URL ')
                if self.remove_punctuation:
                    text = re.sub(r'[^a-zA-Z0-9_]',' ', text, flags = re.M|re.S|re.I)
                if self.remove_numbers:
                    text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', ' NUMBER ', text, flags = re.I|re.S|re.M)
                if self.remove_email:
                    text = convert_email_tags(text)
                if self.remove_stopwords:
                    #words without stemming
                    word_list = remove_stopwords(text)
                if self.stemming:
                    stemmed_list = stemmer(word_list)
                X_transformed.append(stemmed_list)
        return X_transformed
    

In [228]:
from sklearn.base import BaseEstimator,TransformerMixin
import urlextract
import re
from collections import Counter
class CountWords(BaseEstimator,TransformerMixin):
    def __init__(self):
        return None
    def fit(self,X,y = None):
        return self
    def transform(self,X,y = None):
        #create a word vector
        X_wordcounts = []
        for email_word_list in X:
            X_wordcounts.append(Counter(email_word_list))
        #returns a Counter object
        return X_wordcounts

In [232]:
from sklearn.base import BaseEstimator,TransformerMixin
from scipy.sparse import csr_matrix
import numpy as np
class CreateWordVector(BaseEstimator, TransformerMixin):
    def __init__(self,vocabulary_size = 1000):
        self.vocabulary_size = vocabulary_size
    def fit(self,X,y=None):
        total_count = Counter()
        for email in X:
            #each email is a counter object here
            for word,count in email.items():
                total_count[word] += min(count,10)
        self.most_common = total_count.most_common()[:self.vocabulary_size]
        self.vocabulary_ = {word:index + 1 for index,(word,count) in enumerate(self.most_common)}
        return self
    def transform(self,X,y=None):
        rows = []
        cols = []
        occurences = []
        result_array = np.zeros(shape = (len(X),self.vocabulary_size + 1))
        for i,email in enumerate(X):
            for word,count in email.items():
                #getting the index position for the word in most common dictionary. This will be used as the entry to cols
                idx = self.vocabulary_.get(word,0)
                if idx == 0:
                    result_array[i,0] = result_array[i,0] + count
                else:
                    result_array[i,idx] = result_array[i,idx] + count  
        return [result_array,self.most_common]

In [233]:
from sklearn.pipeline import Pipeline

datatransformation = Pipeline([
    ('emailtoword',EmailToWord()),
    ('countwords',CountWords()),
    ('createwordvector',CreateWordVector())
])

In [234]:
res = datatransformation.fit_transform(X_train)

In [236]:
X_train_transformed = res[0]
most_common = res[1]

In [237]:
len(most_common)

1000

In [239]:
#making a dataframe from the data we have obtained of the transformed emails
import pandas as pd
columns = [i[0] for i in most_common]
columns.insert(0,'Not in Vocabulary')
X_train_transformed_df = pd.DataFrame(data = X_train_transformed,columns = columns)

In [242]:
X_train_transformed_df['Result'] = y_train
X_train_transformed_df

,Not in Vocabulary,number,url,list,use,mail,get,one,time,like,...,owner,decis,expert,match,btw,middl,mode,effici,startup,Result
0,11.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,18.0,10.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,22.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,77.0,22.0,0.0,0.0,3.0,2.0,3.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,19.0,17.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,14.0,6.0,4.0,2.0,1.0,2.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2436,43.0,11.0,1.0,1.0,3.0,1.0,1.0,3.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
2437,9.0,1.0,2.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2438,16.0,11.0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [243]:
import matplotlib.pyplot as plt
import seaborn as sns

In [244]:
#drawing histogram for spam email and normal email to see word distributions. 
#words not in vocabulary will be left out and we shall only draw for the most occuring 20 words in each category